# 02-2 스크래핑 사용하기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/hg-da/blob/main/02-2.ipynb"><img src="https://jupyter.org/assets/share.png" width="61" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-da/blob/main/02-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

## 전체 도서의 크기구하기

In [26]:
from bs4 import BeautifulSoup

In [33]:
def get_size_cnt(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[4]
    return ''

In [34]:
get_size_cnt(9791190090018)

'130*198*30mm'

In [35]:
top10_books = books.head(10)

In [38]:
def get_size_cnt2(row):
    isbn = row['isbn13']
    return get_size_cnt(isbn)

In [39]:
size_count = top10_books.apply(get_size_cnt2, axis=1)
print(size_count)

0    130*198*30mm
1    134*200*20mm
2    130*195*15mm
3    133*200*23mm
4    134*195*20mm
5    145*210*30mm
6    137*197*18mm
7                
8    140*210*16mm
9    145*210*20mm
dtype: object


In [40]:
size_count = top10_books.apply(lambda row: get_size_cnt(row['isbn13']), axis=1)

In [42]:
size_count.name = 'size_count'
print(size_count)

0    130*198*30mm
1    134*200*20mm
2    130*195*15mm
3    133*200*23mm
4    134*195*20mm
5    145*210*30mm
6    137*197*18mm
7                
8    140*210*16mm
9    145*210*20mm
Name: size_count, dtype: object


In [43]:
top10_with_size_count = pd.merge(top10_books, size_count,
                                 left_index=True, right_index=True)
top10_with_size_count

,no,ranking,bookname,authors,publisher,publication_year,isbn13,size_count
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,130*198*30mm
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,134*200*20mm
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,130*195*15mm
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,133*200*23mm
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,134*195*20mm
5,6,6,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,145*210*30mm
6,7,7,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,137*197*18mm
7,8,8,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,
8,9,9,선량한 차별주의자,김지혜 지음,창비,2019,9788936477196,140*210*16mm
9,10,9,쇼코의 미소 :최은영 소설,지은이: 최은영,문학동네,2016,9788954641630,145*210*20mm
